<a href="https://colab.research.google.com/github/pranavgupta0001/yorkUCourses/blob/main/eecs4412/eecs4412_phase3_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from sklearn.metrics import cohen_kappa_score, jaccard_score
import scipy.stats as stats


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
!pip install -q mlxtend

In [2]:
!git clone https://github.com/pranavgupta0001/yorkUCourses.git


Cloning into 'yorkUCourses'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 43 (delta 11), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (43/43), 425.43 KiB | 7.60 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
# Load the data
INTC = pd.read_csv('/content/yorkUCourses/eecs4412/data/INTC.csv')
NVDA = pd.read_csv('/content/yorkUCourses/eecs4412/data/NVDA.csv')
AMZN = pd.read_csv('/content/yorkUCourses/eecs4412/data/AMZN.csv')
IBM = pd.read_csv('/content/yorkUCourses/eecs4412/data/IBM.csv')
GOOG = pd.read_csv('/content/yorkUCourses/eecs4412/data/GOOG.csv')
META = pd.read_csv('/content/yorkUCourses/eecs4412/data/META.csv')
MSFT = pd.read_csv('/content/yorkUCourses/eecs4412/data/MSFT.csv')
TSLA = pd.read_csv('/content/yorkUCourses/eecs4412/data/TSLA.csv')
ORCL = pd.read_csv('/content/yorkUCourses/eecs4412/data/ORCL.csv')
AAPL = pd.read_csv('/content/yorkUCourses/eecs4412/data/AAPL.csv')

In [5]:
# Combine the dataframes and add 'Company' column
INTC['Company'] = 'INTC'
NVDA['Company'] = 'NVDA'
AMZN['Company'] = 'AMZN'
IBM['Company'] = 'IBM'
GOOG['Company'] = 'GOOG'
META['Company'] = 'META'
MSFT['Company'] = 'MSFT'
TSLA['Company'] = 'TSLA'
ORCL['Company'] = 'ORCL'
AAPL['Company'] = 'AAPL'

# Concatenate all dataframes
df = pd.concat([INTC, NVDA, AMZN, IBM, GOOG, META, MSFT, TSLA, ORCL, AAPL])

# Keep only necessary columns and sort by date
df = df[['Date', 'Open', 'Close', 'Volume', 'Company']]
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Add new columns for each company
companies = ['INTC', 'NVDA', 'AMZN', 'IBM', 'GOOG', 'META', 'MSFT', 'TSLA', 'ORCL', 'AAPL']
for company in companies:
    df[company] = 0

# Merge the 10 rows of each day
for company in companies:
    df.loc[df['Company'] == company, company] = df.loc[df['Company'] == company].apply(lambda row: row['Volume'] if row['Close'] - row['Open'] > 0 else 0, axis=1)

# Drop unnecessary columns
df = df.drop(columns=['Open', 'Close', 'Volume', 'Company'])

# Aggregate rows by date
df = df.groupby('Date').sum().reset_index()

# Display the updated dataframe
df.head()

,Date,INTC,NVDA,AMZN,IBM,GOOG,META,MSFT,TSLA,ORCL,AAPL
0,2021-06-08,0,0,68334000,5313785,25060000,0,0,0,10726800,74403800
1,2021-06-09,0,0,49110000,5547252,0,0,0,0,0,0
2,2021-06-10,0,28777200,69530000,0,31234000,13240200,24563600,71758800,0,0
3,2021-06-11,0,41630800,0,3596462,0,0,0,0,11066500,53522400
4,2021-06-14,0,32137600,51394000,0,22550000,12250800,19150500,61272000,0,96906500


In [6]:
# Final check
print('Number of columns:', len(df.columns))
print('Number of rows:', len(df))
print('Columns:', df.columns.tolist())

Number of columns: 11
Number of rows: 505
Columns: ['Date', 'INTC', 'NVDA', 'AMZN', 'IBM', 'GOOG', 'META', 'MSFT', 'TSLA', 'ORCL', 'AAPL']


In [8]:
# Preprocess the data
df_binary = df.copy()
for company in companies:
    df_binary[company] = df_binary[company].apply(lambda x: True if x > 0 else False)

# Display the binary matrix
df_binary.head()

,Date,INTC,NVDA,AMZN,IBM,GOOG,META,MSFT,TSLA,ORCL,AAPL
0,2021-06-08,False,False,True,True,True,False,False,False,True,True
1,2021-06-09,False,False,True,True,False,False,False,False,False,False
2,2021-06-10,False,True,True,False,True,True,True,True,False,False
3,2021-06-11,False,True,False,True,False,False,False,False,True,True
4,2021-06-14,False,True,True,False,True,True,True,True,False,True


In [10]:
# Apply the Apriori algorithm
frequent_itemsets = apriori(df_binary[companies], min_support=0.1, use_colnames=True)

# Filter itemsets of size 2 or more
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2]

# Display the 10 most frequent itemsets
frequent_itemsets = frequent_itemsets.sort_values('support', ascending=False)
frequent_itemsets.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
51,0.439604,"(MSFT, AAPL)",2
44,0.423762,"(GOOG, AAPL)",2
26,0.409901,"(NVDA, AAPL)",2
41,0.403960,"(MSFT, GOOG)",2
48,0.401980,"(META, AAPL)",2
40,0.400000,"(META, GOOG)",2
33,0.398020,"(AMZN, AAPL)",2
45,0.392079,"(META, MSFT)",2
23,0.392079,"(NVDA, MSFT)",2
22,0.388119,"(META, NVDA)",2


In [12]:
# Generate association rules with support_only=True
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.1, support_only=True)

# Sort rules by confidence
rules = rules.sort_values('confidence', ascending=False)

# Display the top 5 and bottom 5 rules
print('Top 5 rules by confidence:')
print(rules.head(5))
print('\nBottom 5 rules by confidence:')
print(rules.tail(5))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Top 5 rules by confidence:
  antecedents consequents  antecedent support  consequent support   support  \
0      (MSFT)      (AAPL)                 NaN                 NaN  0.439604   
1      (AAPL)      (MSFT)                 NaN                 NaN  0.439604   
2      (GOOG)      (AAPL)                 NaN                 NaN  0.423762   
3      (AAPL)      (GOOG)                 NaN                 NaN  0.423762   
4      (NVDA)      (AAPL)                 NaN                 NaN  0.409901   

   confidence  lift  leverage  conviction  zhangs_metric  
0         NaN   NaN       NaN         NaN            NaN  
1         NaN   NaN       NaN         NaN            NaN  
2         NaN   NaN       NaN         NaN            NaN  
3         NaN   NaN       NaN         NaN            NaN  
4         NaN   NaN       NaN         NaN            NaN  

Bottom 5 rules by confidence:
      antecedents                     consequents  antecedent support  \
24457      (NVDA)   (AAPL, IBM, META, IN

In [13]:
# Compute the support of each itemset
support = df_binary[companies].mean()

# Generate association rules manually
rules = []
for index, row in frequent_itemsets.iterrows():
    itemset = list(row['itemsets'])
    for i in range(len(itemset)):
        antecedent = itemset[i]
        consequent = itemset[:i] + itemset[i+1:]
        support_antecedent = support[antecedent]
        support_itemset = row['support']
        confidence = support_itemset / support_antecedent
        rules.append((antecedent, consequent, confidence))

# Sort rules by confidence
rules.sort(key=lambda x: x[2], reverse=True)

# Display the top 5 and bottom 5 rules
print('Top 5 rules by confidence:')
for rule in rules[:5]:
    print(f'{rule[0]} -> {rule[1]}: {rule[2]}')
print('\nBottom 5 rules by confidence:')
for rule in rules[-5:]:
    print(f'{rule[0]} -> {rule[1]}: {rule[2]}')

Top 5 rules by confidence:
MSFT -> ['AAPL']: 0.8505747126436781
AMZN -> ['AAPL']: 0.8271604938271606
GOOG -> ['AAPL']: 0.8075471698113207
AAPL -> ['MSFT']: 0.8043478260869564
NVDA -> ['AAPL']: 0.7961538461538461

Bottom 5 rules by confidence:
AAPL -> ['TSLA', 'GOOG', 'META', 'INTC', 'MSFT']: 0.18840579710144925
AAPL -> ['TSLA', 'NVDA', 'GOOG', 'AMZN', 'INTC', 'MSFT']: 0.18478260869565216
AAPL -> ['NVDA', 'GOOG', 'IBM', 'AMZN', 'INTC']: 0.18478260869565216
AAPL -> ['IBM', 'AMZN', 'META', 'INTC']: 0.18478260869565216
AAPL -> ['NVDA', 'IBM', 'META', 'INTC', 'MSFT']: 0.18478260869565216


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Compute measures of interest for the top 5 rules
measures = []
for rule in rules[:5]:
    antecedent, consequent, confidence = rule
    support_antecedent = support[antecedent]
    support_consequent = support[consequent[0]] if len(consequent) == 1 else 0
    support_itemset = frequent_itemsets[frequent_itemsets['itemsets'] == set([antecedent] + consequent)]['support'].values[0]
    lift = support_itemset / (support_antecedent * support_consequent) if support_consequent > 0 else 0
    leverage = support_itemset - support_antecedent * support_consequent
    conviction = (1 - support_consequent) / (1 - confidence) if confidence < 1 else 0
    measures.append((antecedent, consequent, support_itemset, confidence, lift, leverage, conviction))

# Display the measures of interest
for measure in measures:
    print(f'{measure[0]} -> {measure[1]}: support = {measure[2]}, confidence = {measure[3]}, lift = {measure[4]}, leverage = {measure[5]}, conviction = {measure[6]}')

MSFT -> ['AAPL']: support = 0.4396039603960396, confidence = 0.8505747126436781, lift = 1.5563051807429618, leverage = 0.1571375355357318, conviction = 3.0347296268088337
AMZN -> ['AAPL']: support = 0.39801980198019804, confidence = 0.8271604938271606, lift = 1.5134639470388263, leverage = 0.13503382021370453, conviction = 2.6236209335219254
GOOG -> ['AAPL']: support = 0.42376237623762375, confidence = 0.8075471698113207, lift = 1.4775772491112933, leverage = 0.13696696402313496, conviction = 2.3562415065035904
AAPL -> ['MSFT']: support = 0.4396039603960396, confidence = 0.8043478260869564, lift = 1.5563051807429618, leverage = 0.1571375355357318, conviction = 2.469526952695268
NVDA -> ['AAPL']: support = 0.4099009900990099, confidence = 0.7961538461538461, lift = 1.4567307692307692, leverage = 0.1285168120772473, conviction = 2.224546983000186


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# Compute measures of interest for the top 5 rules
measures = []
for rule in rules[:5]:
    antecedent, consequent, _ = rule

    # Compute correlation
    correlation = df_binary[antecedent].corr(df_binary[consequent[0]])

    # Compute odds ratio
    contingency_table = pd.crosstab(df_binary[antecedent], df_binary[consequent[0]])
    odds_ratio, _ = stats.fisher_exact(contingency_table)

    # Compute kappa
    kappa = cohen_kappa_score(df_binary[antecedent], df_binary[consequent[0]])

    # Compute interest
    interest = support_itemset / (support_antecedent * support_consequent)

    # Compute Jaccard index
    jaccard = jaccard_score(df_binary[antecedent], df_binary[consequent[0]])

    measures.append((antecedent, consequent, correlation, odds_ratio, kappa, interest, jaccard))

# Display the measures of interest
for measure in measures:
    print(f'{measure[0]} -> {measure[1]}: correlation = {measure[2]}, odds ratio = {measure[3]}, kappa = {measure[4]}, interest = {measure[5]}, Jaccard index = {measure[6]}')

MSFT -> ['AAPL']: correlation = 0.6316481737231883, odds ratio = 20.02849002849003, kappa = 0.6305255953364329, interest = 1.4567307692307692, Jaccard index = 0.7047619047619048
AMZN -> ['AAPL']: correlation = 0.5428742544562912, odds ratio = 11.932380952380953, kappa = 0.5382505333739713, interest = 1.4567307692307692, Jaccard index = 0.6320754716981132
GOOG -> ['AAPL']: correlation = 0.5509316793160198, odds ratio = 12.046805819101834, kappa = 0.5504037817608824, interest = 1.4567307692307692, Jaccard index = 0.654434250764526
AAPL -> ['MSFT']: correlation = 0.6316481737231884, odds ratio = 20.02849002849003, kappa = 0.6305255953364329, interest = 1.4567307692307692, Jaccard index = 0.7047619047619048
NVDA -> ['AAPL']: correlation = 0.5165361271527159, odds ratio = 9.962264150943396, kappa = 0.5154922931739541, interest = 1.4567307692307692, Jaccard index = 0.6291793313069909


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
